<a href="https://colab.research.google.com/github/Zaqzxcswsde/itmo-applied-statistics/blob/main/stat_tests.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

TODO: переделать парсинг, чтобы работал с дробями и переменными

In [37]:
#@title Imports { display-mode: "form" }
from pandas import Series, DataFrame
import pandas as pd
import numpy as np
from os import listdir
# from google.colab import drive
#drive.mount('/content/drive')
# import matplotlib.pyplot as plt
# %matplotlib inline
import itertools
from itertools import product
from scipy import special
# как можно скачать файл напрямую из гуглдиска сюда в директорию /content
# !gdown 1Xtw5-jqrDJmh318t1jtKHvOJXxMrB9XF
from math import factorial
from decimal import *
import sympy
from itertools import product
from math import fsum
from fractions import Fraction
import scipy
from scipy import stats
from sympy import integrate, Symbol
from math import pi,sqrt
from numpy import exp

---
---
---

# Неделя 1

---
## Function Definitions

In [38]:
#@title Получить произведение всех чисел в листе
def list_product(array):
  s = 1
  for i in array:
    s *= i
  return s

In [39]:
#@title Функции комбинаторики

# формула C из комбинаторики с повторениями и без
def comb(n,k, *, repetition=0):
  if repetition: return comb(n+k-1,k)
  else: return Decimal(factorial(n))/Decimal(Decimal(factorial(k))*Decimal(factorial(n-k)))

# 4 основные функции комбинаторики объединены в одну, повторения есть? порядок важен? n? k?
def prob(n,k,o,r):
  if not o: return Decimal(comb(n,k,repetition=r))
  elif r: return Decimal(n)**Decimal(k)
  else: return Decimal(special.perm(n,k))

# упрощённый доступ напрямую к C из n по k
def cnk(n,k):
  return Decimal(prob(n,k,0,0))

In [40]:
#@title Конкатенация листа в строчку
def conc(lst):
  c = ''
  for i in lst:
    c+=str(i)
  return c

In [41]:
#@title Является ли число факториалом другого?
def isfactorial(x):
  if not isinstance(x,int) or x<1: return -1
  i=0
  while factorial(i)<x:
    i+=1
  if factorial(i)==x: return i
  else: return -1

In [42]:
#@title Перевод числа в 6-ти-ричную систему
def tosix(n):
    if n == 0:
        return ['0']
    digits = []
    while n:
        digits.append(str(int(n % 6)))
        n //= 6
    return digits[::-1]

In [43]:
#@title Является ли лист уникальным? Не работает, если там хранятся другие листы (и прочие нехешитруемые типы)
def isunique(lst):
  return len(list(lst)) == len(list(set(list(lst))))

---
## Code itself

In [44]:
# сколько из всех вариантов пар произведений чисел от 1 до 4 целочисленно делятся на три 
list(map(lambda a: a % 3, map(lambda a: a[0]*a[1], itertools.product(np.arange(1,5),np.arange(1,5))))).count(0)

7

In [45]:
# у скольки из всех вариантов пар произведений чисел от 1 до 4 сумма превышает произведение 
len([1 for a in itertools.product(np.arange(1,5),np.arange(1,5)) if a[0]+a[1]>a[0]*a[1]])

7

In [46]:
# сколькими способами можно выбрать из n пар детей двих так, чтобы они были из разных пар?
pair_count=3
4*sum(range(1,pair_count))

12

In [47]:
# или так
all_possible_combinations=list(map(list,list(product(range(6),repeat=2))))
all_possible_pair_variations=[[x[0]//2,x[1]//2] for x in all_possible_combinations]
apc_with_rep_1=list(map(set,all_possible_combinations))
apc_with_rep_2=[x for x in apc_with_rep_1 if len(x)>1]
apc_without_rep=list(map(list,(set(map(tuple,apc_with_rep_2)))))
apc_with_rep_pairs_1=[[x[0]//2,x[1]//2] for x in list(map(list,apc_without_rep))]
apc_with_rep_pairs_2=list(map(set,apc_with_rep_pairs_1))
apc_without_rep_pairs=[x for x in apc_with_rep_pairs_2 if len(x)==2]
len(apc_without_rep_pairs)

12

In [48]:
# из цифр от 0 до 5 составили декартово проиведение длиной 4, в скольких вариантах нет повторений цифр?
# способ 1
len([x for x in map(set,product(range(0,6),repeat=4)) if len(x)==4])

360

In [49]:
# способ 2
sum(list(map(isunique,[''.join(tosix(x)).zfill(4) for x in range(0,6**4)])))

360

In [50]:
# или так! 6 вариантов на первую, 5 на вторую и тд до четвёртой
6*5*4*3

360

---
---
# Неделя 2


---
## Function definitions

In [51]:
#@title Парсинг таблиц из опенеду в датафрейм/серию/лист
# Работает только с числовыми значениями, не работает с переменными или дробями - TODO
def OpenEduTexToArrayOfArrays(full_tex,dec=True):
  def is_float(element):
    try:
        float(element)
        return True
    except ValueError:
        return False
  full_tex=full_tex.replace(r'\hline','\n')      
  full_tex=list(map(lambda a: a.split(),full_tex.split('\n')))
  for i,x in enumerate(full_tex):
    full_tex[i]=[y for y in x if is_float(y)]
  full_tex = [x for x in full_tex if len(x)>0]
  if dec:
    for i,x in enumerate(full_tex):
      full_tex[i]=list(map(lambda a: Fraction(str(a)),x))
  return full_tex
def ArrayOfArraysWithIndexsAndColumnsToDataFrame(arrofarr):
  if len(arrofarr)<=2: raise RuntimeError('don\'t use this function for series!')
  if len(arrofarr[0])<len(arrofarr[1]): arrofarr[0] = [''] + arrofarr[0]
  npaoa = np.array(arrofarr)
  return DataFrame(npaoa[1:,1:],columns=npaoa[0,:][1:],index=npaoa[:,0][1:],dtype="object")
def TTDFOVAP(full_tex,dec=True): # TexToDataFrameOrValsAndProbs
  arrofarr=OpenEduTexToArrayOfArrays(full_tex,dec=dec)
  if len(arrofarr)==1: return arrofarr[0]
  elif len(arrofarr)==2: return (arrofarr[0],arrofarr[1])
  else: return ArrayOfArraysWithIndexsAndColumnsToDataFrame(arrofarr)

In [52]:
#@title Медиана распределения случайной величины
def myMedian(vals, probs):
  m_1 = [i for i,_ in enumerate(probs) if ((sum(probs[:i+1])>0.5) and (sum(probs[i:])>0.5))]
  m_1_e = [vals[v] for v in m_1]
  m_2 = m_1_e[0] if len(m_1_e)==1 else (max(m_1_e)+min(m_1_e))/2
  return m_2

In [53]:
#@title Проверка величин на независимость, !!!!!внутри DataFrame'а нужны Decimal'ы/Fractions'ы!!!!!
def CheckIndependence(df):
  xsi=df.sum(axis=0) # this produces series of decimals, as expected
  # eta=df.sum(axis=1) # this instead produces series of floats, not decimals - presumably pandas bug
  eta=Series({k:v.sum() for k,v in df.iterrows()})
  # xsi=xsi.apply(lambda a: Decimal(str(a)))
  # eta=eta.apply(lambda a: Decimal(str(a)))
  df2 = pd.DataFrame(columns=df.columns, index=df.index)
  for i in df.columns:
    for j in df.index:
      df2[i][j]=xsi[i]*eta[j]
  return df.equals(df2)

In [54]:
#@title Математическое ожидание распределения случайной величины
def myExpectedValue(vals, probs):
  vals = np.asarray(vals)
  probs = np.asarray(probs)
  return (vals * probs).sum()

In [55]:
#@title Совместное распределение -> мат.ожидание их произведения
def TablToExpectedOfMult(df):
  df2=DataFrame(columns=df.columns,index=df.index)
  for i in df2.columns:
    for j in df2.index:
      df2[i][j]=i*j
  ss={}
  for i in df2.columns:
    for j in df2.index:
      if df2[i][j] in ss.keys():
        ss[df2[i][j]]+=df[i][j]
      else:
        ss[df2[i][j]]=df[i][j]
  return sum(np.asarray(list(ss.keys()))*np.asarray(list(ss.values())))

In [56]:
#@title Распределение случайной величины -> дисперсия
def MyVariance(vals,probs):
  ev = myExpectedValue(vals,probs)
  vals2=[(x-ev)**2 for x in vals]
  disp = myExpectedValue(vals2,probs)
  return disp

In [57]:
#@title Распределение случайной величины -> дисперсия 2
def MyVariance2(vals,probs):
  ev = myExpectedValue(vals,probs)
  vals2=[x**2 for x in vals]
  disp = myExpectedValue(vals2,probs)
  return disp-ev**2

In [58]:
#@title Совместное распределение -> мат.ожидание обеих - сначала по столбцам, потом по строкам
def MyJointDistributionToExpectedOfBoth(df):
  xsi,eta=MyJointDistributionToMarginalDistribution(df)
  xsi_ev=myExpectedValue(xsi.index,xsi.values)
  eta_ev=myExpectedValue(eta.index,eta.values)
  return (xsi_ev,eta_ev)

In [59]:
#@title Совметное распределение -> маргинальное распределение
def MyJointDistributionToMarginalDistribution(df):
  xsi=df.sum(axis=0)
  eta=Series({k:v.sum() for k,v in df.iterrows()})
  return (xsi,eta)

In [60]:
#@title Маргинальное распределение / дисперсия -> СКО
def MyStandardDeviation(vals=None,probs=None,var=None):
  if not var: return decimal_from_fraction(MyVariance(vals,probs))**Decimal(0.5)
  else: return decimal_from_fraction(var)**Decimal(0.5)

In [61]:
#@title Совметное распределение -> ковариация
def MyCovariance(df):
  ksi_ev, eta_ev = MyJointDistributionToExpectedOfBoth(df)
  return TablToExpectedOfMult(df) - ksi_ev*eta_ev

In [62]:
#@title Совметное распределение -> дисперсия обеих
def MyJointDistributionToVarianceOfBoth(df):
  xsi, eta = MyJointDistributionToMarginalDistribution(df)
  xsi_var = MyVariance(xsi.index,xsi.values)
  eta_var = MyVariance(eta.index,eta.values)
  return (xsi_var,eta_var)

In [63]:
#@title Совметное распределение -> коэффицент корелляции (Пирсона)
def MyPearsonCorrelationCoefficient(df):
  cov = MyCovariance(df)
  if cov == Decimal('0'): return Decimal('0')
  xsi_var, eta_var = MyJointDistributionToVarianceOfBoth(df)
  if xsi_var == eta_var: return cov/xsi_var
  xsi_dev = MyStandardDeviation(var=xsi_var)
  eta_dev = MyStandardDeviation(var=eta_var)
  return decimal_from_fraction(cov)/(xsi_dev*eta_dev)

In [64]:
#@title Decimal -> Fraction
def decimal_from_fraction(frac):
  if isinstance(frac,Decimal): return frac
  return frac.numerator / Decimal(frac.denominator)

In [65]:
#@title Схема бернули в нунжном мне виде
def MyBernoulli(n,k,p,q=None): # n - всего, k - нужно удач, p - вероятность удачи, q - вероятность неудачи
  n = Decimal(n)
  k = Decimal(k)
  p = Decimal(str(p))
  if not q: q = Decimal(1-p)
  return Decimal(cnk(n,k))*(Decimal(p)**Decimal(k))*(Decimal(q)**(n-k))

In [66]:
#@title НЕ ИСПОЛЬЗОВАТЬ! Лобовая реализация вычисления мат. ожидания и дисперсии случайной величины схемы Бернулли
def BAD_MyXsiBernoulli(n,p,q=None):
  if not q: q = 1-p
  vals = range(0,n+1)
  probs = [MyBernoulli(n,x,Decimal(str(p)),Decimal(str(q))) for x in range(0,n+1)]
  ev=myExpectedValue(vals,probs)
  var=MyVariance(vals,probs)
  return (ev,var)

In [67]:
#@title Нормальная реализация вычисления мат. ожидания и дисперсии случайной величины схемы Бернулли
def MyXsiBernoulli(n,p,q=None):
  if not q: q = 1 - p
  return (n*p,n*p*q)

In [68]:
#@title Локальная теорема Муавра-Лапласа (n,p,A,B)->val
def MyLocML(n,p,A,B,r=4):
  A,B=min(A,B),max(A,B)
  q=1-p
  # koef=1/(sqrt(2*pi)*sqrt(n*p*q))
  val = round(sum(exp(-(((k-n*p)/(sqrt(n*p*q)))**2)/2) for k in range(A,B+1))*(1/(sqrt(2*pi)*sqrt(n*p*q))),4)
  return val

In [69]:
#@title Теорема Пуассона (n,p,A,B)->(val,(leftErr,rightErr))
def MyPoisson(n,p,A,B,r=4):
  A,B=min(A,B),max(A,B)
  mu=n*p
  val = round(sum(((Decimal(Decimal(mu)**k)/Decimal(factorial(k))) * (exp(Decimal(-mu)))) for k in range(A,B+1)),r)
  err = Decimal(min(Decimal(str(p)),n*(Decimal(str(p))**2)))
  leftErr = val-err if val-err>0 else 0
  rightErr = val+err if val+err<1 else 1
  return (val,(leftErr,rightErr))

In [70]:
#@title Интегральная теорема Муавра-Лапласа (n,p,A,B)->(val,err)
def MyIntML(n,p,A,B,r=4):
  A,B=min(A,B),max(A,B)
  q=1-p
  y=(B-n*p)/sqrt(n*p*q)
  z=(A-n*p)/sqrt(n*p*q)
  val = round(scipy.stats.norm.cdf(y)-scipy.stats.norm.cdf(z),r)
  err = round(1/(p*q*sqrt(n)),r)
  return (val,err)

In [71]:
#@title Пуассон + Локальный М-Л
def MyPML(n,p,A,B,r=4):
  A,B=min(A,B),max(A,B)
  return (MyPoisson(n,p,A,B,r),MyLocML(n,p,A,B,r))

In [72]:
#@title Бернулли для k между A и B
def MyABBernoulli(n,p,A,B,r=4):
  A,B=min(A,B),max(A,B)
  return round(sum(MyBernoulli(n,k,p) for k in range(A,B+1)),r)

---
## Code itself

### First Half

In [73]:
# Количество орлов/решек при трёх подбрасываниях монеты
list(map(lambda a: a.count(0), itertools.product([0,1],repeat=3)))
list(map(lambda a: a.count(1), itertools.product([0,1],repeat=3)))

[0, 1, 1, 2, 1, 2, 2, 3]

In [74]:
# Нахождение подбором одного неизвестного значения случайного распределения по известному математическому ожиданию и остальному распределению
for x in range(-1,14):
  vals = list(map(Decimal,f'-1 & 1 & 4 & {x} & 13'.split(' & ')))
  probs  = list(map(Decimal,'0.1 & 0.15 & 0.4 & 0.2 & 0.15'.split(' & ')))
  if myExpectedValue(vals, probs)==Decimal('5.2'): print(x)

8


In [75]:
# Автоматическое нахождение уравнения для математического ожидания, хотя можно было просто посмотреть...
vals='3 5 7'.split(' ')
probs='p1 p2 0.3'.split(' ')
expr=sympy.sympify('+'.join([f'{x1}*{x2}' for x1,x2 in zip(vals,probs)]))
expr

3*p1 + 5*p2 + 2.1

In [76]:
# Тест парсинга в vals,probs
full_tex=r"""\begin{array}{  c | c | c | c | c| c | c| c | c  }
\xi & -4 & -2 & 0 & 3 & 4 & 6 & 7 & 9  \\ \hline
\mathsf{P} & 0.1 & 0.15 & 0.05 & 0.15 & 0.1 & 0.05 & 0.25 & 0.15
\end{array}"""
vals,probs=TTDFOVAP(full_tex,dec=0)
vals,probs

(['-4', '-2', '0', '3', '4', '6', '7', '9'],
 ['0.1', '0.15', '0.05', '0.15', '0.1', '0.05', '0.25', '0.15'])

In [77]:
# Тест парсинга в датафрейм
full_tex=r"""\begin{array}{  c | c | c | c | c  }
\eta \backslash \xi & -4 & -2 & 0 & 3  \\ \hline
2 & 0.02 & 0.02 & 0.06 & 0.1 \\ \hline
3 & 0.03 & 0.03 & 0.09 & 0.15 \\ \hline
5 & 0.02 & 0.02 & 0.06 & 0.1 \\ \hline
10 & 0.03 & 0.03 & 0.09 & 0.15
\end{array}"""
df = TTDFOVAP(full_tex)

In [78]:
df

,-4,-2,0,3
2,1/50,1/50,3/50,1/10
3,3/100,3/100,9/100,3/20
5,1/50,1/50,3/50,1/10
10,3/100,3/100,9/100,3/20


In [79]:
# Проверка независимости

# arr=np.array([[0.02,0.02,0.06,0.1],[0.03,0.03,0.09,0.15],[0.02,0.02,0.06,0.1],[0.03,0.03,0.09,0.15]])
# arr2 = np.vectorize(lambda a: Decimal(str(a)))(arr)
# df = pd.DataFrame(arr2,columns=[-4,-2,0,3], index=[2,3,5,10])

# full_tex=r"""\begin{array}{  c | c | c | c | c  }
# \eta \backslash \xi & -4 & -2 & 0 & 3  \\ \hline
# 2 & 0.02 & 0.02 & 0.06 & 0.1 \\ \hline
# 3 & 0.03 & 0.03 & 0.09 & 0.15 \\ \hline
# 5 & 0.02 & 0.02 & 0.06 & 0.1 \\ \hline
# 10 & 0.03 & 0.03 & 0.09 & 0.15
# \end{array}"""
# df = TTDFOVAP(full_tex)

CheckIndependence(df)

True

In [80]:
# Нахождение ожидания произведения
TablToExpectedOfMult(df)

Fraction(477, 100)

In [81]:
# проверка работы нахождения дисперсии
vals=[0,100]
probs=[99/100,1/100]
MyVariance(vals,probs) # MyVariance2(vals,probs)

99.0

In [82]:
# проверка работы нахождения СКО
xsi,eta = MyJointDistributionToMarginalDistribution(df)
xsi_vals = xsi.index
xsi_probs = xsi.values
eta_vals = eta.index
eta_probs = eta.values

In [83]:
MyStandardDeviation(xsi_vals,xsi_probs)

Decimal('2.385372088375312569654571324')

In [84]:
MyStandardDeviation(eta_vals,eta_probs)

Decimal('3.226453160980335694543128270')

In [85]:
# тест ковариации !
MyCovariance(df)

Fraction(0, 1)

In [86]:
# тест коэффициента корелляции
MyPearsonCorrelationCoefficient(df)

Decimal('0')

In [87]:
# Моделирование всего-всего-всего на примере двухкратного бросания игрального кубика
omega = list(itertools.product(np.arange(1,7),repeat=2))

# xsi - количество выпавших единиц
# eta - количество выпавших шестёрок

# xsi - 0/1/2
# eta - 0/1/2
omega_xsi_eta = np.array([[x.count(1),x.count(6)] for x in omega])
omega_xsi, omega_eta = omega_xsi_eta.T

# получаем маргинальное распределение
marg_xsi = np.unique(omega_xsi, return_counts=True)
marg_eta = np.unique(omega_eta, return_counts=True)

# преобразовываем в серию + починим вероятность
xsi = Series(list(map(lambda a: Fraction(a,len(omega)),marg_xsi[1])), index=marg_xsi[0])
eta = Series(list(map(lambda a: Fraction(a,len(omega)),marg_eta[1])), index=marg_eta[0])

# сразу получим совместное распределение
possible_vars = list(map(list,list(itertools.product([0,1,2],repeat=2))))
numbs = ['']*len(possible_vars)
for i,x in enumerate(possible_vars):
  numbs[i] = sum([1 for y in omega_xsi_eta if list(y)==x])
to_df = [[x[0][0],x[0][1],x[1]] for x in list(zip(possible_vars,numbs))]

df = DataFrame(index=[0,1,2],columns=[0,1,2]) 
for x in to_df:
  df[x[0]][x[1]]=Fraction(x[2],len(omega))

In [88]:
df

,0,1,2
0,4/9,2/9,1/36
1,2/9,1/18,0
2,1/36,0,0


In [89]:
# Математические ожидания кси/эта
MyJointDistributionToExpectedOfBoth(df)

(Fraction(1, 3), Fraction(1, 3))

In [90]:
# Дисперсии кси/эта
MyJointDistributionToVarianceOfBoth(df)

(Fraction(5, 18), Fraction(5, 18))

In [91]:
# Маетматическое ожидание произведения кси/эта
TablToExpectedOfMult(df)

Fraction(1, 18)

In [92]:
# Ковариация
MyCovariance(df)

Fraction(-1, 18)

In [93]:
# Коэффициент корелляции
MyPearsonCorrelationCoefficient(df)

Fraction(-1, 5)

In [94]:
full_tex=r"""\begin{array}{  c | c | c | c | c  }
\xi & -4 & -2 & 0 & 3  \\ \hline
\mathsf{P} & 0.1 & 0.1 & 0.3 & 0.5
\end{array}"""
vals,probs=TTDFOVAP(full_tex)

In [95]:
MyVariance(vals,probs)

Fraction(569, 100)

In [96]:
full_tex=r"""\begin{array}{  c | c | c | c  }
\eta \backslash \xi & 2 & 3 & 5  \\ \hline
-1 & 0.1 & 0.3 & 0.2 \\ \hline
1 & 0.1 & 0.05 & 0 \\ \hline
4 & 0 & 0.15 & 0.1
\end{array}"""
df=TTDFOVAP(full_tex)

In [97]:
df

,2,3,5
-1,1/10,3/10,1/5
1,1/10,1/20,0
4,0,3/20,1/10


In [98]:
MyCovariance(df)

Fraction(9, 50)

In [99]:
MyPearsonCorrelationCoefficient(df)

Decimal('0.07664850422701901346493947032')

In [100]:
BAD_MyXsiBernoulli(10,0.75)

(Decimal('7.50000000000000000000'), Decimal('1.875000000000000000000000000'))

In [101]:
MyXsiBernoulli(10,0.75)

(7.5, 1.875)

In [102]:
# Просто для красоты, даже без заголовка и перенесения в блок с функциями :)
def MyDescribe(full_tex):
  df=TTDFOVAP(full_tex)
  print('MarginalDistributions: ', MyJointDistributionToMarginalDistribution(df)[0],MyJointDistributionToMarginalDistribution(df)[1],'------',sep='\n')
  print('ExpectedValues: ', MyJointDistributionToExpectedOfBoth(df),'------',sep='\n')
  print('Variances: ', MyJointDistributionToVarianceOfBoth(df),'------',sep='\n')
  print('Covariance: ', MyCovariance(df),'------',sep='\n')
  print('CorrelationCoefficient: ', MyPearsonCorrelationCoefficient(df), sep='\n')

In [103]:
full_tex=r"""\begin{array}{   c | c | c | c | c  } \eta \backslash \xi  & 156 & 168 & 173 & 193  \\ \hline 2 & 0.05 & 0.04 & 0.29 & 0.17 \\ \hline 4 & 0.07 & 0.08 & 0.19 & 0.11 \end{array}"""
MyDescribe(full_tex)

MarginalDistributions: 
156     3/25
168     3/25
173    12/25
193     7/25
dtype: object
2    11/20
4     9/20
dtype: object
------
ExpectedValues: 
(Fraction(4399, 25), Fraction(29, 10))
------
Variances: 
(Fraction(88074, 625), Fraction(99, 100))
------
Covariance: 
-361/250
------
CorrelationCoefficient: 
-0.1222547506417396250214209794


### Second Half

In [104]:
# по теореме Пуассона + погрешность, по локальной теореме Муавра-Лапласаp
n=100
A=0
B=3
p=0.02
MyPML(n,p,A,B)

((Decimal('0.8571'), (Decimal('0.8371'), Decimal('0.8771'))), 0.8293)

In [105]:
# по интегральной теореме Муавра-Лапласа + погрешность
p=0.13
n=20000
A=0
B=2650
MyIntML(n,p,A,B)

(0.8534, 0.0625)

In [106]:
# w2p2t1 - week 2 part 2 task 1
n=1600
p=0.0026
A=5
B=1600

(MyABBernoulli(n,p,A,B),MyPoisson(n,p,A,B),MyIntML(n,p,A,B))

(Decimal('0.4024'),
 (Decimal('0.4024'), (Decimal('0.3998'), Decimal('0.4050'))),
 (0.34, 9.6404))

In [107]:
# w2p2t2
n=4300
p=0.6
A=2535
B=2618

(MyABBernoulli(n,p,A,B),MyPoisson(n,p,A,B),MyIntML(n,p,A,B))

(Decimal('0.8062'), (Decimal('0.5908'), (0, 1)), (0.8009, 0.0635))

---
---
# Неделя 3